# Exploring Heating Problems in Manhattan

To learn how to query data with SparkSQL, in this notebook we will investigate New York City's 311 Complaints data, which is available as part of NYC Open Data. 

In particular we are interested in Heating Complaints within the Manhattan Borough. In this notebook, we will be exploring the data, using Spark's SQL module and the visualization tool Brunel.

During the months of October to May, residents of NYC can call 311 to report that a building doesn't have enough heat or hot water. In the remaining months, June to September, complaints can be made that heating has been left on. 

There may be a number of different factors that contribute to heating complaints, we will select a few of the features available in our data to see if they suggest any correlation.


## Table of contents
1. [Install Libraries](#install)
2. [Load and read the data](#loaddataset)
3. [Spark SQL exploration](#SSexploration)
4. [Summary](#summary)

## 1. Install Libraries<a class="anchor" id="install"></a>

First, make sure you install the libraries you need for visualization.

In [1]:
import brunel
# Brunel will be used to visualize data later on


## Load and read the data<a class="anchor" id="loaddataset"></a>

Begin by downloading a 311 Complaints data set for New York City from NYC Open Data to your local file system.

To download the data set:

1. Click [NYC Open Data](https://data.cityofnewyork.us/browse?q=311+complaints&provenance=official) and select a data set, for example, the 311 Service Requests 2015 data set.
2. Download the file.


Then load the data file to the notebook. 

To load the CSV file:

1. Click the **Find and Add Data** icon on the notebook action bar. and then browse the CSV file you downloaded or drag it into your notebook sidebar. The file is loaded to your object storage and also appears in the Data Assets section of your project.

2. Click in the empty code cell below and then click the **Insert to code** link below the file and select **Insert SparkSession DataFrame**. 

## Read the data
The data has been saved in the Object Store; once inserted, the code generated by **Insert to code** provides the appropriate credentials to be able to access and read our data. Once you've inserted the code, change the `df_data_1` variable to `nyc311DF` before you run the cell.

In [2]:
# use this cell to load your data and credentials by using Insert to code -> Insert SparkSession DataFrame



[Row(Unique Key='29608104', Created Date='01/01/2015 12:00:00 AM', Closed Date='01/08/2015 12:00:00 AM', Agency='HPD', Agency Name='Department of Housing Preservation and Development', Complaint Type='HEAT/HOT WATER', Descriptor='ENTIRE BUILDING', Location Type='RESIDENTIAL BUILDING', Incident Zip='11212', Incident Address='1803 PITKIN AVENUE', Street Name='PITKIN AVENUE', Cross Street 1='MOTHER GASTON BOULEVARD', Cross Street 2='CHRISTOPHER AVENUE', Intersection Street 1=None, Intersection Street 2=None, Address Type='ADDRESS', City='BROOKLYN', Landmark=None, Facility Type='N/A', Status='Closed', Due Date=None, Resolution Description='The Department of Housing Preservation and Development inspected the following conditions. Violations were issued. Information about specific violations is available at www.nyc.gov/hpd.', Resolution Action Updated Date='01/08/2015 12:00:00 AM', Community Board='16 BROOKLYN', Borough='BROOKLYN', X Coordinate (State Plane)='1010208', Y Coordinate (State Pl

We are explicitly telling Spark that our data is of CSV format, has a header and that we wish for the documents schema to be inferred. This will be saved as a DataFrame named nyc311DF.

In [3]:
nyc311DF.count()

2282348

Now let's see what columns we have so we can select a few to investigate.

In [4]:
nyc311DF.printSchema()

root
 |-- Unique Key: string (nullable = true)
 |-- Created Date: string (nullable = true)
 |-- Closed Date: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Agency Name: string (nullable = true)
 |-- Complaint Type: string (nullable = true)
 |-- Descriptor: string (nullable = true)
 |-- Location Type: string (nullable = true)
 |-- Incident Zip: string (nullable = true)
 |-- Incident Address: string (nullable = true)
 |-- Street Name: string (nullable = true)
 |-- Cross Street 1: string (nullable = true)
 |-- Cross Street 2: string (nullable = true)
 |-- Intersection Street 1: string (nullable = true)
 |-- Intersection Street 2: string (nullable = true)
 |-- Address Type: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Landmark: string (nullable = true)
 |-- Facility Type: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Due Date: string (nullable = true)
 |-- Resolution Description: string (nullable = true)
 |-- Resolution Action

## Spark SQL exploration<a class="anchor" id="SSexploration"></a>
SparkSQL is a powerful tool allowing users a familiar and relatively intuitive way to explore the data. In order to refer to the data within an SQL query, it needs to be stored as a view. The following query creates a temporary view named nyc311ct.

In [5]:
nyc311DF.createOrReplaceTempView("nyc311ct")

In [6]:
spark.sql("select distinct Borough from nyc311ct").show()

+-------------+
|      Borough|
+-------------+
|  Unspecified|
|       QUEENS|
|     BROOKLYN|
|        BRONX|
|    MANHATTAN|
|STATEN ISLAND|
+-------------+



Let's find the complaint type with the most complaints in Manhattan.
Note that we are calling the cache function, this means that when the next action is called ("show", "count", etc.) it will store the DataFrame nyc311Agr_df in memory for much quicker retrieval in the future. However, this must be small enough to fit.

In [7]:
nyc311Agr_df = spark.sql("select `Complaint Type` as Complaint_Type, count(`Unique Key`) as Complaint_Count "
                            "from nyc311ct where Borough = 'MANHATTAN' "
                            "group by `Complaint Type` order by Complaint_Count desc").cache()

In [8]:
nyc311Agr_df.show(4)

+--------------------+---------------+
|      Complaint_Type|Complaint_Count|
+--------------------+---------------+
|      HEAT/HOT WATER|          54103|
| Noise - Residential|          46769|
|               Noise|          27347|
|Noise - Street/Si...|          22351|
+--------------------+---------------+
only showing top 4 rows



Let's get a visual representation of the data within nyc311Agr_df. We are creating a bubble chart, where the size of the bubble represents the number of complaints. The complaint type is assigned a color, and, if large enough, the bubble is labeled, else the type displayed when hovered over.

In [9]:
custom_frame = nyc311Agr_df.toPandas()
custom_frame.head(4)

,Complaint_Type,Complaint_Count
0,HEAT/HOT WATER,54103
1,Noise - Residential,46769
2,Noise,27347
3,Noise - Street/Sidewalk,22351


In [10]:
%brunel data('custom_frame') bubble size(Complaint_Count) color(Complaint_Type) label(Complaint_Type) legends(none) tooltip(Complaint_Type)


<IPython.core.display.Javascript object>

How does the number of complaints vary by zip code? Let's remove any data points where a zip code hasn't been provided and filter to those that are of type 'HEAT/HOT WATER'.
Note: If just exploring the data, where you do not intend to re-use the resulting DataFrame, you can just use Spark SQL with the function "show" without assigning it to a variable.

In [11]:
spark.sql("select `Incident Zip` as Zip, count(*) as ZipHeatingCnt " 
          "from nyc311ct " 
          "where `Complaint Type` = 'HEAT/HOT WATER' and `Incident Zip` <> '' group by `Incident Zip`").show()

+-----+-------------+
|  Zip|ZipHeatingCnt|
+-----+-------------+
|11205|         1586|
|11236|         1134|
|10309|           22|
|11106|          700|
|11218|         1675|
|10452|         4469|
|11428|          120|
|11237|         1444|
|11379|          149|
|11364|          157|
|11109|            1|
|11249|          749|
|10012|          688|
|11001|            8|
|11385|         1289|
|11238|         1913|
|10039|          967|
|11427|          174|
|11367|          477|
|10010|          496|
+-----+-------------+
only showing top 20 rows



Similarly, if you wish to use the result of these queries for future queries but do not require the data as a DataFrame, you can create a table directly from the query as follows:

In [12]:
spark.sql("select `Incident Zip` as Zip, count(*) as ZipHeatingCnt "  
          "from nyc311ct " 
          "where `Complaint Type` = 'HEAT/HOT WATER' and `Incident Zip` <> '' group by `Incident Zip`").createOrReplaceTempView("zipHeatingCnt")

Let's see which date and zip codes had the most complaints. The 'Created Date' field includes a time, therefore we are using the "split" function to just use the date. We are also limiting the data to only heat/hot water complaints, and for the month of March.

In [13]:
spark.sql("select split(`Created Date`, ' ')[0] as Incident_Date, `Incident Zip` as Incident_Zip, "
          "count(`Unique Key`) as HeatingComplaintCount "
          "from nyc311ct where `Complaint Type` = 'HEAT/HOT WATER' and `Incident Zip` <> '' "
          "and split(split(`Created Date`, ' ')[0], '/')[0] = '03' "
          "group by split(`Created Date`, ' ')[0], `Incident Zip` order by HeatingComplaintCount desc limit 50").show()

+-------------+------------+---------------------+
|Incident_Date|Incident_Zip|HeatingComplaintCount|
+-------------+------------+---------------------+
|   03/05/2015|       10040|                   59|
|   03/06/2015|       10040|                   57|
|   03/01/2015|       10040|                   56|
|   03/28/2015|       10040|                   55|
|   03/05/2015|       11226|                   53|
|   03/02/2015|       10467|                   53|
|   03/31/2015|       10040|                   48|
|   03/05/2015|       10458|                   47|
|   03/07/2015|       10040|                   47|
|   03/06/2015|       11205|                   46|
|   03/07/2015|       10468|                   46|
|   03/18/2015|       10462|                   46|
|   03/20/2015|       10467|                   43|
|   03/02/2015|       10468|                   42|
|   03/23/2015|       10453|                   42|
|   03/01/2015|       10468|                   41|
|   03/21/2015|       10468|   

## Summary<a class="anchor" id="summary"></a>

This concludes the tutorial on how to query using SparkSQL. 

### Author

**Rohan Vaidyanathan** is a Senior Offering Manager in IBM. He is responsible for Spark and Hadoop offerings on IBM Cloud. He has been part of the Big Data and Analytics product portfolio in IBM since 2012 and has focused on building products that help users be more productive with open source analytics technologies.

<hr>
Copyright © 2017, 2018 IBM. This notebook and its source code are released under the terms of the MIT License.


<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>
